# Text-based Information Retrieval
## Assignment Part I - Warmup

*Assignment part 1 (10 points out of 100 total)*

>Your task will be to run several analogy solving models with several
different representations on the benchmarking analogy dataset and report your findings. Focus on the
following questions:
1. Is the choice of the analogy model important? Which representations work better with which analogy
models?
2. Is dimensionality of the representation important when using GloVe vectors?
3. What is the computational complexity of the analogy models given the pre-trained vectors?
4. What are the typical errors?


### Practical Info

Information about linguistic regularities in Word Predictions:
http://www.marekrei.com/blog/linguistic-regularities-word-representations/

List of questions to ask:
http://word2vec.googlecode.com/svn/trunk/questions-words.txt

Pretrained vector sets:
* Word2Vec: https://code.google.com/archive/p/word2vec/
* GloVe: http://nlp.stanford.edu/projects/glove/

## Code

Note: This is written using Python 3 - there may be small differences if using another version

Load in required libraries, load in Word2Vec model

In [1]:
# Import modules
from gensim import models
#import numpy as np
import logging

# Set up logger that logs (works in jupyter 3!) in console and outputs in file
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='mylog.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

### Define functions

Because we need to use different analogy models and need to calculate the recall value, functions would be useful...
[Gensims' implementation](https://github.com/piskvorky/gensim/blob/develop/gensim/models/word2vec.py)

The different analogy models are explained here: http://www.marekrei.com/blog/linguistic-regularities-word-representations/


**Model 1** (addition model)

a : b is c : ?  (Or, a to b is c to [...], with a, b and c are word vectors)
>1. Compute the vector c - a + b
>2. Find the closest vector

In [2]:
def analogy_model1(a, b, c, model): 
    result = model.most_similar(positive=[c, b], negative=[a], topn=1)
    return result[0]

**Model 2** (Multiplication model)

a : b is c : d  (Or, a to b is c to d, with a, b and c are word vectors)
>d = argmax(cos(d',c)*cos(d',b)/(cos(d'a)+e))
>
>e = 0.001 to avoid division by zero

In [3]:
def analogy_model2(a, b, c, model):
    result = model.most_similar_cosmul(positive=[c, b], negative=[a], topn=1)
    return result[0]

**Rec@ll1**

Each analogy model that we test should report its performance as a *Recall@1* metric
>[Recall](https://en.wikipedia.org/wiki/Precision_and_recall#Recall) in information retrieval is the fraction of the documents that are relevant to the query that are successfully retrieved.

>![alt text](recall_formula.png "Recall@1")

In [4]:
def recall_analogy_model(questions, analogy_model, model):
    right_count = 0 
    total_count = 0
    skipped_count = 0

    with open(questions, 'r') as file:
        for line in file:
            if line[0] != ':' :   # Ignore the lines that start with a ':', they indicate semantic/syntactic relation categories
                total_count += 1
                words = line.split() # Split the different words
                try:
                    result_text = analogy_model(words[0], words[1], words[2], model)                 
                    if result_text[0] == words[3]:
                        right_count += 1
                except KeyError, e:
                    skipped_count += 1
                
    # Return the recall number, the total right, the total count and the skipped count
    # The recall number is calculated based on
    recall = float(right_count) / float(total_count)
    recall_ignore_skipped = float(right_count) / float(total_count - skipped_count)
    return float('%.5f'% recall), float('%.5f'% recall_ignore_skipped), float(right_count), float(total_count), float(skipped_count)

In [69]:
import sys
import os
import itertools
from numpy import exp, log, dot, zeros, outer, random, dtype, float32 as REAL,\
    uint32, seterr, array, uint8, vstack, fromstring, sqrt, newaxis,\
    ndarray, empty, sum as np_sum, prod, ones, ascontiguousarray

from gensim import utils, matutils  # utility fnc for pickling, common scipy operations etc
from six import iteritems, itervalues

logger = logging.getLogger(__name__)

# Modified gensim accuracy for recall analogy
def recall_gensim_model(questions, analogy_model, model):
    #def accuracy(self, questions, restrict_vocab=30000, most_similar=most_similar):
    """
    Compute accuracy of the model. `questions` is a filename where lines are
    4-tuples of words, split into sections by ": SECTION NAME" lines.
    See https://code.google.com/p/word2vec/source/browse/trunk/questions-words.txt for an example.
    The accuracy is reported (=printed to log and returned as a list) for each
    section separately, plus there's one aggregate summary at the end.
    Use `restrict_vocab` to ignore all questions containing a word whose frequency
    is not in the top-N most frequent words (default top 30,000).
    This method corresponds to the `compute-accuracy` script of the original C word2vec.
    """
    restrict_vocab = 30000
    ok_vocab = dict(sorted(iteritems(model.vocab),
                           key=lambda item: -item[1].count)[:restrict_vocab])
    ok_index = set(v.index for v in itervalues(ok_vocab))

    sections, section = [], None
    for line_no, line in enumerate(utils.smart_open(questions)):
        # TODO: use level3 BLAS (=evaluate multiple questions at once), for speed
        line = utils.to_unicode(line)
        if line.startswith(': '):
            # a new section starts => store the old section
            if section:
                sections.append(section)
                #self.log_accuracy(section)
            section = {'section': line.lstrip(': ').strip(), 'correct': [], 'incorrect': []}
        else:
            if not section:
                raise ValueError("missing section header before line #%i in %s" % (line_no, questions))
            try:
                a, b, c, expected = [word.lower() for word in line.split()]  # TODO assumes vocabulary preprocessing uses lowercase, too...
            except:
                logger.info("skipping invalid line #%i in %s" % (line_no, questions))
            if a not in ok_vocab or b not in ok_vocab or c not in ok_vocab or expected not in ok_vocab:
                logger.debug("skipping line #%i with OOV words: %s" % (line_no, line.strip()))
                continue

            ignore = set(model.vocab[v].index for v in [a, b, c])  # indexes of words to ignore
            predicted = None
            # find the most likely prediction, ignoring OOV words and input words
            #sims = analogy_model(a, b, c, model)
            sims = model.most_similar(positive=[b, c], negative=[a], topn=False)
            for index in matutils.argsort(sims, reverse= True):
                if index in ok_index and index not in ignore:
                    predicted = model.index2word[index]
                    #if predicted != expected:
                    #    logger.debug("%s: expected %s, predicted %s", line.strip(), expected, predicted)
                    #break
            if predicted == expected:
                section['correct'].append((a, b, c, expected))
            else:
                section['incorrect'].append((a, b, c, expected))
    if section:
        # store the last section, too
        sections.append(section)
        #self.log_accuracy(section)
    '''
    total = {
        'section': 'total',
        'correct': sum((s['correct'] for s in sections), []),
        'incorrect': sum((s['incorrect'] for s in sections), []),
    }
    '''
    print sections
    correct = sum(len(s['correct']) for s in sections)
    print "Correct: ", correct
    total = correct + sum(len(s['incorrect']) for s in sections)
    print "Total: ", total
    
    recall = float(correct) / float(total)
    return float('%.5f'% recall)
'''
    self.log_accuracy(total)
    sections.append(total)
    return sections
    
'''

'\n    self.log_accuracy(total)\n    sections.append(total)\n    return sections\n    \n'

### Word2Vec


In [71]:
def recallOfAnalogyModel(questions, r_model, a_model):
    """
    questions are file path for the test file. In this case, 'questions-words.txt'
    r_model is representation model of word vector (word2vec, GloVe)
    a_model is analogy model (1, 2). 1 stands for findAnalogy_model1, 2 stands for findAnalogy_model2
    """
    count_correct = 0 # counter for number of correct result
    count_total = 0 # count total number of questions
    
    if a_model == 1:
        with open(questions, 'r') as ifile:
            for line in ifile:
                if line[0] != ':' :
                    count_total += 1
                    line_sp = line.split()
                    result_text = analogy_model1(line_sp[0], line_sp[1], line_sp[2], r_model)                 
                    if result_text[0] == line_sp[3]:
                        count_correct += 1
    elif a_model == 2:
        with open(questions, 'r') as ifile:
            for line in ifile:
                if line[0] != ':' :
                    count_total += 1
                    line_sp = line.split()
                    result_text = analogy_model2(line_sp[0], line_sp[1], line_sp[2], r_model)
                    if result_text[0] == line_sp[3]:
                        count_correct += 1
    else:
        raise ValueError("invalid analogy model")
    recall = float(count_correct) / float(count_total)
    return float('%.4f'% recall)

In [76]:
print "Word2Vec - addition model", recallOfAnalogyModel('data/questions-words.txt', w2v_model, 1)


Word2Vec - addition model

IndexError: list index out of range

In [5]:
# Load Googles' pre-trained Word2Vec vector set
# Note: This will take a lot of memory and can take a while.
# Note II: Depending on your RAM, do not load all models at the same time

w2v_model = models.Word2Vec.load_word2vec_format('data/GoogleNews-vectors-negative300.bin.gz', binary=True)

w2v_model.init_sims(replace=True) # Trim unneeded model memory = use (much) less RAM.

In [6]:
# Run for analogy models
print "Word2Vec - addition model", recall_analogy_model('data/questions-words-test.txt', analogy_model1, w2v_model)
#print ("Word2Vec - multiplication model", recall_analogy_model('data/questions-words.txt', analogy_model2, w2v_model))

Word2Vec - addition model 0.78142


In [ ]:
print "Word2Vec - addition model", recall_analogy_model('data/questions-words.txt', analogy_model1, w2v_model)


In [67]:
analogy_model1('father', 'mother', 'brothers', w2v_model)

[(u'sisters', 0.8290482759475708)]

In [60]:
somevar = [{'correct': [(u'boy', u'girl', u'brother', u'sister'),
   (u'boy', u'girl', u'brothers', u'sisters'),
   (u'boy', u'girl', u'dad', u'mom'),
   (u'boy', u'girl', u'father', u'mother')], 'incorrect': [], 'section': u'capital-common-countries'},
 {'correct': [1,2,3,4,5,6], 'incorrect': [1,2,3], 'section': u'capital-world'}]

print somevar

correct = sum(len(s['correct']) for s in somevar)
incorrect = sum(len(s['incorrect']) for s in somevar)
total = correct + incorrect
print "Correct:   ", correct
print "Incorrect: ", incorrect
print "Total:     ", total
print "Procent:   ", (float(correct) / float(total))


[{'incorrect': [], 'section': u'capital-common-countries', 'correct': [(u'boy', u'girl', u'brother', u'sister'), (u'boy', u'girl', u'brothers', u'sisters'), (u'boy', u'girl', u'dad', u'mom'), (u'boy', u'girl', u'father', u'mother')]}, {'incorrect': [1, 2, 3], 'section': u'capital-world', 'correct': [1, 2, 3, 4, 5, 6]}]
Correct:    10
Incorrect:  3
Total:      13
Procent:    0.769230769231


### GloVe

Gloves' vector model is constructed differently than Word2Vec. But, once constructed, the vector model format is very similar to the Word2Vec model. However, there are some small differences. The answer to adapt Glove to Word2Vec is found [here](https://groups.google.com/forum/#!topic/gensim/0_SeYGVAL78) and the code [here](https://github.com/manasRK/glove-gensim).

In [5]:
import smart_open
import os.path

def glove2word2vec(glove_filename):
    def get_info(glove_filename): 
        num_lines = sum(1 for line in smart_open.smart_open(glove_filename))
        dims = glove_filename.split('.')[2].split('d')[0] # file name contains the number of dimensions
        return num_lines, dims
    
    def prepend_info(infile, outfile, line): # Function to prepend lines using smart_open
        with open(infile, 'r', encoding="utf8") as original: data = original.read()
        with open(outfile, 'w', encoding="utf8") as modified: modified.write(line + '\n' + data)
        return outfile
    
    word2vec_filename = glove_filename[:-3] + "word2vec.txt"
    if os.path.isfile(word2vec_filename):
        model = models.Word2Vec.load_word2vec_format(word2vec_filename)
    else:
        num_lines, dims = get_info(glove_filename)
        gensim_first_line = "{} {}".format(num_lines, dims)
        model_file = prepend_info(glove_filename, word2vec_filename, gensim_first_line)
        model = models.Word2Vec.load_word2vec_format(model_file)
    
    model.init_sims(replace = True)  # normalize all word vectors
    return model

In [7]:
# Load GloVes' pre-trained model
# These vectors are stored in a plain text - vector dimensionality 50, 100, 200 and 300
# only the vectors pre-trained on Wikipedia.
glove50d_model = glove2word2vec('data/glove.6B.50d.txt')

INFO:gensim.models.word2vec:loading projection weights from data/glove.6B.50d.word2vec.txt
INFO:gensim.models.word2vec:loaded (400000, 50) matrix from data/glove.6B.50d.word2vec.txt
INFO:gensim.models.word2vec:precomputing L2-norms of word weight vectors


In [46]:
print ("GloVe50d - addition model", recall_analogy_model('data/questions-words.txt', analogy_model1, glove50d_model))
print ("GloVe50d - multiplication model", recall_analogy_model('data/questions-words.txt', analogy_model2, glove50d_model))

KeyError: "word 'Baghdad' not in vocabulary"

In [8]:
glove100d_model = glove2word2vec('data/glove.6B.100d.txt')
glove200d_model = glove2word2vec('data/glove.6B.200d.txt')
glove300d_model = glove2word2vec('data/glove.6B.300d.txt')

INFO:gensim.models.word2vec:loading projection weights from data/glove.6B.100d.word2vec.txt
INFO:gensim.models.word2vec:loaded (400000, 100) matrix from data/glove.6B.100d.word2vec.txt
INFO:gensim.models.word2vec:precomputing L2-norms of word weight vectors
INFO:gensim.models.word2vec:loading projection weights from data/glove.6B.200d.word2vec.txt
INFO:gensim.models.word2vec:loaded (400000, 200) matrix from data/glove.6B.200d.word2vec.txt
INFO:gensim.models.word2vec:precomputing L2-norms of word weight vectors


MemoryError: 

In [57]:
print ("GloVe100d - addition model", recall_analogy_model('data/questions-words.txt', analogy_model1, glove100d_model))
#print ("GloVe100d - multiplication model", recall_analogy_model('data/questions-words.txt', analogy_model2, glove100d_model))
#print ("GloVe200d - addition model", recall_analogy_model('data/questions-words.txt', analogy_model1, glove200d_model))
#print ("GloVe200d - multiplication model", recall_analogy_model('data/questions-words.txt', analogy_model2, glove200d_model))
#print ("GloVe300d - addition model", recall_analogy_model('data/questions-words.txt', analogy_model1, glove300d_model))
#print ("GloVe300d - multiplication model", recall_analogy_model('data/questions-words.txt', analogy_model2, glove300d_model))

KeyError: "word 'Baghdad' not in vocabulary"


## More info on Word2Vec

Also, More info on how to use gensim can be found in [this tutorial](http://rare-technologies.com/word2vec-tutorial/).

Gensim accepts the bin format, but if you want the txt format.. Gensim can transform it:

>model = gensim.models.Word2Vec.load_word2vec_format('path/to/GoogleNews-vectors-negative300.bin', binary=True)
>model.save_word2vec_format('path/to/GoogleNews-vectors-negative300.txt', binary=False)

Once the model is loaded, Gensim supports a lot of out-of-the-box functionality.

In [11]:
# Get top-most similar word
w2v_model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)

[('queen', 0.7118192315101624)]

In [12]:
# Find the word that doesnt fit in the row
w2v_model.doesnt_match("breakfast cereal dinner lunch".split())

'cereal'

In [13]:
# Get the similarity between two words
w2v_model.similarity('woman', 'man')

0.76640122344103145

In [14]:
# Get the raw numpy vector of a certain word
w2v_model['computer']

array([  4.08137441e-02,  -7.64330178e-02,   4.67502922e-02,
         8.05143863e-02,  -3.46916839e-02,   8.23695585e-02,
        -5.00895977e-02,   3.15378942e-02,   7.68040493e-02,
         1.81806684e-02,   1.39137767e-02,  -9.32223070e-03,
         9.09033418e-03,  -6.08495846e-02,  -9.92516056e-03,
         3.69178876e-02,  -2.41172127e-02,   7.01254383e-02,
         6.49309605e-02,  -6.19626865e-02,  -4.15558144e-02,
         5.67682087e-02,  -1.76820919e-04,   3.65468524e-02,
         6.41888902e-02,   9.91356559e-04,   3.39496173e-02,
         2.46737637e-02,   1.35427425e-02,  -2.63434183e-02,
        -5.56551069e-02,  -4.60082218e-02,  -8.64509344e-02,
         9.32223070e-03,  -4.73068431e-02,  -1.20957099e-01,
        -8.38536993e-02,   4.97185625e-02,   1.39137767e-02,
        -1.38210189e-02,  -4.30399515e-02,   7.42068142e-02,
         3.71034071e-02,   4.82344255e-02,   2.50447989e-02,
         2.63434183e-02,   3.89585760e-03,   6.67861328e-02,
        -6.41888902e-02,

Or, Gensim supports the same format as Googles' question words.

In [39]:
# Gensim supports the same evaluation set as Google does
w2v_model.accuracy('data/questions-words-test.txt')

[{'correct': [], 'incorrect': [], 'section': u'capital-common-countries'},
 {'correct': [], 'incorrect': [], 'section': u'capital-world'},
 {'correct': [(u'boy', u'girl', u'brother', u'sister'),
   (u'boy', u'girl', u'brothers', u'sisters'),
   (u'boy', u'girl', u'dad', u'mom'),
   (u'boy', u'girl', u'father', u'mother'),
   (u'boy', u'girl', u'grandfather', u'grandmother'),
   (u'boy', u'girl', u'grandson', u'granddaughter'),
   (u'boy', u'girl', u'groom', u'bride'),
   (u'boy', u'girl', u'he', u'she'),
   (u'boy', u'girl', u'his', u'her'),
   (u'boy', u'girl', u'husband', u'wife'),
   (u'boy', u'girl', u'king', u'queen'),
   (u'boy', u'girl', u'man', u'woman'),
   (u'boy', u'girl', u'nephew', u'niece'),
   (u'boy', u'girl', u'prince', u'princess'),
   (u'boy', u'girl', u'son', u'daughter'),
   (u'boy', u'girl', u'sons', u'daughters'),
   (u'boy', u'girl', u'uncle', u'aunt'),
   (u'brother', u'sister', u'brothers', u'sisters'),
   (u'brother', u'sister', u'dad', u'mom'),
   (u'brother